# Importing Libaries

In [ ]:
import numpy as np
import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
#  write the  code for mounting your Google Drive
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
#write the code for unzipping the data and storing it in Google Drive
!unzip /content/traffic_data.zip

Streaming output truncated to the last 5000 lines.
  inflating: traffic_Data/DATA/26/026_0028.png  
 extracting: traffic_Data/DATA/26/026_0029.png  
  inflating: traffic_Data/DATA/26/026_0030.png  
  inflating: traffic_Data/DATA/26/026_0031.png  
  inflating: traffic_Data/DATA/26/026_0032.png  
 extracting: traffic_Data/DATA/26/026_0033.png  
  inflating: traffic_Data/DATA/26/026_0034.png  
  inflating: traffic_Data/DATA/26/026_0035.png  
 extracting: traffic_Data/DATA/26/026_0036.png  
  inflating: traffic_Data/DATA/26/026_0037.png  
  inflating: traffic_Data/DATA/26/026_0038.png  
  inflating: traffic_Data/DATA/26/026_0039.png  
  inflating: traffic_Data/DATA/26/026_0040.png  
  inflating: traffic_Data/DATA/26/026_0041.png  
  inflating: traffic_Data/DATA/26/026_0042.png  
  inflating: traffic_Data/DATA/26/026_0043.png  
  inflating: traffic_Data/DATA/26/026_0044.png  
  inflating: traffic_Data/DATA/26/026_0045.png  
  inflating: traffic_Data/DATA/26/026_0046.png  
  inflating: traff

In [ ]:
dataset_path = os.listdir('traffic_data')

class_labels = []

for item in dataset_path:
  if item == 'DATA':
    all_classes = os.listdir('traffic_data'+'/'+item)


    for image in all_classes:
      all_images = os.listdir('traffic_data'+'/'+item+'/'+image)
      for im in all_images:
        class_labels.append((image,str('traffic_data'+'/'+item)+'/'+image+'/'+im))


In [ ]:
# Build a dataframe
df = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])
print(df.head())
print(df.tail())

  Labels                                image
0     13  traffic_data/DATA/13/013_1_0007.png
1     13    traffic_data/DATA/13/013_0002.png
2     13  traffic_data/DATA/13/013_1_0004.png
3     13  traffic_data/DATA/13/013_1_0009.png
4     13    traffic_data/DATA/13/013_0001.png
     Labels                                image
4165     56  traffic_data/DATA/56/056_1_0010.png
4166     56    traffic_data/DATA/56/056_0010.png
4167     56  traffic_data/DATA/56/056_1_0034.png
4168     56  traffic_data/DATA/56/056_1_0026.png
4169     56    traffic_data/DATA/56/056_0032.png


In [ ]:
# Let's check how many samples for each category are present
print("Total number of images in the dataset: ", len(df))

label_count = df['Labels'].value_counts()
print(label_count)

Total number of images in the dataset:  4170
28    446
54    324
3     260
5     194
55    162
35    156
7     152
30    150
16    142
11    138
17    130
14    128
26    126
0     118
56    110
24    100
4      98
12     96
43     82
2      80
6      78
10     70
37     58
50     56
29     44
31     42
49     42
1      40
36     40
13     36
52     36
39     34
42     32
40     32
44     30
38     30
27     28
34     26
45     24
15     22
20     18
41     18
46     18
22     18
32     14
23     14
47     12
21     12
48     10
8       8
51      8
18      8
57      6
33      4
19      4
53      2
9       2
25      2
Name: Labels, dtype: int64


In [ ]:
import cv2
def load_images_from_folder(folder,target_size=(224, 224)):
  images = []
  for filename in os.listdir(folder):
    img_path = os.path.join(folder,filename)
    img = cv2.imread(img_path)
    img = cv2.resize(img,target_size)
    images.append(img)
  return images

def process_dataset(data_path):
  class_labels = []
  images = []

  for class_label in os.listdir(dataset_path):
    class_folder = os.path.join(dataset_path,class_label)
    if os.path.isdir(class_folder):
      class_labels.append(class_label)

      class_images = load_images_from_folder(class_folder)
      images.extend(class_images)

  return class_labels,images


dataset_path = "/content/traffic_data/DATA"
class_labels, images = process_dataset(dataset_path)

In [ ]:
images = np.array(images)
images = images.astype('float32')/255.0
images.shape

(4170, 224, 224, 3)

In [ ]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y=df['Labels'].values


from keras.utils import to_categorical

y_int = [int(label) for label in y]

y_encoded = to_categorical(y_int)



In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images,y = shuffle(images,y_encoded,random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, y, test_size=0.05, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(3961, 224, 224, 3)
(3961, 58)
(209, 224, 224, 3)
(209, 58)


### Training the model on EfficientNetB0

### Transfer learning from pre-trained weights
Here we initialize the model with pre-trained ImageNet weights,and we fine-tune it on our own dataset.
The first step to transfer learning is to freeze all layers and train only the top layers. For this step, a relatively large learning rate (1e-2) can be used.

EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

include_top=False means we don't want to use classification layer(Dense from the pretrained network)

In [ ]:
import cv2
import numpy as np
from matplotlib.pyplot import imread
from matplotlib.pyplot import imshow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
import numpy as np
import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os


NUM_CLASSES = 58
IMG_SIZE = 224

In [ ]:
def build_model(num_classes):
  inputs = layers.Input(shape=(IMG_SIZE,IMG_SIZE,3))

  x = inputs

  model = EfficientNetB0(include_top=False,weights='imagenet',input_tensor=x)

  model.trainable = False

  x = layers.GlobalAveragePooling2D(name='avg_pool')(model.output)
  x = layers.BatchNormalization()(x)

  top_dropout_rate = 0.2

  x = layers.Dropout(top_dropout_rate,name='top_dropout')(x)
  outputs = layers.Dense(NUM_CLASSES,activation='softmax',name="pred")(x)

  model = tf.keras.Model(inputs, outputs, name="EfficientNet")
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
  model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
  return model



In [ ]:
model = build_model(num_classes=NUM_CLASSES)

16705208/16705208 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "EfficientNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, 224, 224, 3)          0         ['normalization[0][

In [ ]:
epochs = 20
model.fit(train_x, train_y, epochs=epochs, verbose=2)

Epoch 1/20
124/124 - 369s - loss: 5.0027 - accuracy: 0.0573 - 369s/epoch - 3s/step
Epoch 2/20
124/124 - 344s - loss: 4.0319 - accuracy: 0.0795 - 344s/epoch - 3s/step
Epoch 3/20
124/124 - 329s - loss: 3.6999 - accuracy: 0.0932 - 329s/epoch - 3s/step
Epoch 4/20
124/124 - 339s - loss: 3.6272 - accuracy: 0.0987 - 339s/epoch - 3s/step
Epoch 5/20
124/124 - 336s - loss: 3.5998 - accuracy: 0.1045 - 336s/epoch - 3s/step
Epoch 6/20
124/124 - 335s - loss: 3.5833 - accuracy: 0.1053 - 335s/epoch - 3s/step
Epoch 7/20
124/124 - 336s - loss: 3.5810 - accuracy: 0.1063 - 336s/epoch - 3s/step
Epoch 8/20
124/124 - 335s - loss: 3.5838 - accuracy: 0.1070 - 335s/epoch - 3s/step
Epoch 9/20
124/124 - 337s - loss: 3.5746 - accuracy: 0.1055 - 337s/epoch - 3s/step
Epoch 10/20
124/124 - 334s - loss: 3.5798 - accuracy: 0.1073 - 334s/epoch - 3s/step
Epoch 11/20
124/124 - 333s - loss: 3.5738 - accuracy: 0.1053 - 333s/epoch - 3s/step
Epoch 12/20
124/124 - 332s - loss: 3.5836 - accuracy: 0.0962 - 332s/epoch - 3s/step
E

In [ ]:
efficientnet_accuracy = model.evaluate(test_x,test_y)
print("The EfficientNet loss :",efficientnet_accuracy[0])
print("The EfficientNet Accuracy :",efficientnet_accuracy[1])

7/7 [==============================] - 19s 2s/step - loss: 3.6439 - accuracy: 0.0670
The EfficientNet loss : 3.6438629627227783
The EfficientNet Accuracy : 0.06698564440011978
